In [9]:
%load_ext autoreload
%autoreload
import argparse
import copy
import json
import os
import sys

sys.path.append("../")
sys.path.append("../src")

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pandas as pd
from IPython import embed
from matplotlib import pyplot as plt
from torch.autograd import Variable


%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload
import src
from src.i3dense import I3DenseNet
from src.i3res import I3ResNet
from src.evaluator import Evaluator
from src.mit_data import MITDataset
from src.train import train, validation_loss
from src.video_handler import VideoViewer
from src.label_handler import LabelHandler
from src.run import Run
from src.analyze import ResultAnalyzer, plot_learning_curve
import experiment.resnet2d_top20 as resnet2d
import experiment.resnet3d_top20 as resnet3d

## some setup

In [11]:
run_3d = Run("../experiment/resnet3d_top20/run_0001")
run_3d_binary = Run("../experiment/resnet3d_binary/normal_lr_1218/")
run_3d_binary_lowlr = Run("../experiment/resnet3d_binary/low_lr_1218/")
run_2d_low = Run("../experiment/resnet2d_top20/low_lr_run2/")
run_2d_normal = Run("../experiment/resnet2d_top20/normal_lr_run2/")
run_2d_binary = Run("../experiment/resnet2d_binary/run_1215/")
run_cifar = Run("../experiment/cifar10/test1217_lowlr/")
mlb = resnet2d.mlb

In [12]:
model = resnet3d.model

In [16]:
def average_infration(module):
    def average_conv(sub_module):
        for key in sub_module._modules:
            child_module = sub_module._modules[key]
            if isinstance(child_module, nn.modules.conv.Conv3d):
                setattr(sub_module, key, make_averaged_conv(child_module))
            else:
                average_conv(child_module)
    average_conv(module)

In [17]:
def make_averaged_conv(conv_layer):
    new_layer = nn.Conv3d(
            conv_layer.in_channels,
            conv_layer.out_channels,
            conv_layer.kernel_size,
            padding=conv_layer.padding,
            dilation=conv_layer.dilation,
            stride=conv_layer.stride)
    
    new_weight = new_layer.weight.data
    original_weight = conv_layer.weight.data.numpy()
    
    time_dim = original_weight.shape[2]
    middle_idx = conv_layer.kernel_size[0] // 2
    
    weight_2d = original_weight[:,:,middle_idx,:,:]
    weight_2d = np.expand_dims(weight_2d,axis=2)
    weight_2d = np.repeat(weight_2d,3,axis=2)
    weight_2d = weight_2d / time_dim
    
    new_layer.weight = nn.Parameter(torch.from_numpy(weight_2d))
    new_layer.bias = conv_layer.bias
    return new_layer

In [15]:
model.conv1.weight

Parameter containing:
tensor([[[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

          [[ 0.0133,  0.0147, -0.0154,  ..., -0.0409, -0.0430, -0.0708],
           [ 0.0041,  0.0058,  0.0149,  ...,  0.0022, -0.0209, -0.0385],
           [ 0.0223,  0.0236,  0.0161,  ...,  0.1028,  0.0626,  0.0520],
           ...,
           [-0.0009,  0.0278, -0.0101,  ..., -0.1272, -0.0766,  0.0078],
           [ 0.0036,  0.0480,  0.0621,  ...,  0.0243, -0.0337, -0.0157],
           [-0.0800, -0.0322, -0.0178,  ...,  0.0354,  0.0224,  0.0017]],

          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.

In [18]:
average_infration(model)
model

I3ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck3d(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
    

In [19]:
model.conv1.weight

Parameter containing:
tensor([[[[[ 0.0044,  0.0049, -0.0051,  ..., -0.0136, -0.0143, -0.0236],
           [ 0.0014,  0.0019,  0.0050,  ...,  0.0007, -0.0070, -0.0128],
           [ 0.0074,  0.0079,  0.0054,  ...,  0.0343,  0.0209,  0.0173],
           ...,
           [-0.0003,  0.0093, -0.0034,  ..., -0.0424, -0.0255,  0.0026],
           [ 0.0012,  0.0160,  0.0207,  ...,  0.0081, -0.0112, -0.0052],
           [-0.0267, -0.0107, -0.0059,  ...,  0.0118,  0.0075,  0.0006]],

          [[ 0.0044,  0.0049, -0.0051,  ..., -0.0136, -0.0143, -0.0236],
           [ 0.0014,  0.0019,  0.0050,  ...,  0.0007, -0.0070, -0.0128],
           [ 0.0074,  0.0079,  0.0054,  ...,  0.0343,  0.0209,  0.0173],
           ...,
           [-0.0003,  0.0093, -0.0034,  ..., -0.0424, -0.0255,  0.0026],
           [ 0.0012,  0.0160,  0.0207,  ...,  0.0081, -0.0112, -0.0052],
           [-0.0267, -0.0107, -0.0059,  ...,  0.0118,  0.0075,  0.0006]],

          [[ 0.0044,  0.0049, -0.0051,  ..., -0.0136, -0.0143, -0.